In [10]:
import sys
import os
sys.path.append(os.path.abspath('autoformer'))
from autoformer.model import Autoformer
import numpy as np
import pandas as pd
import tensorflow as tf
from keras import layers
from keras.models import Model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

In [11]:
def create_time_windows(data, target, window_size, horizon=1):
    X_windows = []
    y_windows = []
    for i in range(len(data) - window_size - horizon + 1):
        X_windows.append(data[i:i+window_size])  # این باید یک sequence باشد
        y_windows.append(target[i+window_size:i+window_size+horizon])
    return np.array(X_windows), np.array(y_windows)

In [12]:
df= pd.read_csv('data/ETTh1.csv')
df['date']= pd.to_datetime(df['date'])
df['timestamp'] = df['date'].astype('int')
df = df.set_index(['date'])
df.head()

,HUFL,HULL,MUFL,MULL,LUFL,LULL,OT,timestamp
date,,,,,,,,
2016-07-01 00:00:00,5.827,2.009,1.599,0.462,4.203,1.340,30.531000,1467331200000000000
2016-07-01 01:00:00,5.693,2.076,1.492,0.426,4.142,1.371,27.787001,1467334800000000000
2016-07-01 02:00:00,5.157,1.741,1.279,0.355,3.777,1.218,27.787001,1467338400000000000
2016-07-01 03:00:00,5.090,1.942,1.279,0.391,3.807,1.279,25.044001,1467342000000000000
2016-07-01 04:00:00,5.358,1.942,1.492,0.462,3.868,1.279,21.948000,1467345600000000000


In [13]:
X= df.drop(columns=['OT'])
X= X.values
y= df['OT'].values

In [14]:
scaler= StandardScaler()
X_scaled = scaler.fit_transform(X)
y_scaled = scaler.fit_transform(y.reshape(-1, 1)).flatten()

In [15]:
window_size = 24  # last 24 hour
horizon = 1  # forecast next hour

X_windows, y_windows = create_time_windows(X_scaled, y_scaled, window_size, horizon)

if len(y_windows.shape) == 1:
    y_windows = y_windows.reshape(-1, 1)


In [16]:
x_train, x_test, y_train, y_test= train_test_split(X_windows, y_windows, train_size=0.8, shuffle=False)
x_valid, x_test, y_valid, y_test= train_test_split(x_test, y_test, train_size=0.5, shuffle=False)

In [17]:
input_shape = (window_size, X.shape[1])
inputs= layers.Input(shape=input_shape)
x= Autoformer(d_out= 1,
            d_model= 16,
            n_heads= 4,
            conv_filter= 16,
            num_decoder=1,
            num_encoder=1)(inputs)

model= Model(inputs= inputs, outputs= x)      
model.compile(optimizer='adam', loss='mse')
model.summary()  

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 24, 7)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ autoformer_1 (Autoformer)       │ (None, 24, 1)          │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [18]:
model.fit(
    x_train,
    y_train,
    validation_data=(x_valid, y_valid),
    shuffle=False,
    epochs=100,
)

Epoch 1/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 30s 40ms/step - loss: 1.1464 - val_loss: 0.3928
Epoch 2/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 2.0394 - val_loss: 0.4586
Epoch 3/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 1.5606 - val_loss: 0.7056
Epoch 4/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 1.9718 - val_loss: 0.6328
Epoch 5/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 1.8542 - val_loss: 0.7074
Epoch 6/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 1.9884 - val_loss: 0.5579
Epoch 7/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 1.6390 - val_loss: 0.8143
Epoch 8/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 1.3145 - val_loss: 0.4721
Epoch 9/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 1.2417 - val_loss: 0.4232
Epoch 10/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 1.1976 - val_loss: 0.5400
Epoch 11/100
435/435 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step - loss: 1.1663 - val_loss: 0.4149
Epoch 12/100
435/435 ━━━━━━━━

In [19]:
# Evaluate the model accuracy on the validation set.
score = model.evaluate(x_test, y_test)

55/55 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1098
